### About the Dataset

In [ ]:

import pandas as pd

# Load your Excel file into a pandas DataFrame
file_path = "Loan Default Prediction Dataset_exported.csv"
raw_df = pd.read_csv(file_path)

# Display basic information about the raw dataset
print("Raw Dataset Overview:")
print(raw_df.info())

# Display summary statistics for numerical columns
print("\nSummary Statistics for Numerical Columns:")
print(raw_df.describe())

# Display the first few rows of the raw dataset
print("\nFirst Few Rows of the Raw Dataset:")
print(raw_df.head())

# Data Preprocessing

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def preprocess_data(df):
    # Drop rows with missing values (you can customize this based on your dataset)
    df = df.dropna()

    # Drop 'LoanID' if present
    if 'LoanID' in df.columns:
        df = df.drop('LoanID', axis=1)

    # Separate features and target variable
    X = df.drop('Default', axis=1)
    y = df['Default']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Get the original column names for numeric and categorical features
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object']).columns

    # Create transformers for numerical and categorical features
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Combine transformers using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    # Apply preprocessing to training set
    X_train_preprocessed = preprocessor.fit_transform(X_train)

    # Apply the same preprocessing to testing set
    X_test_preprocessed = preprocessor.transform(X_test)

    return X_train_preprocessed, X_test_preprocessed, y_train, y_test



In [ ]:
# Data Preprocessing
file_path = "Loan Default Prediction Dataset_exported.csv"
df = pd.read_csv(file_path)
X_train, X_test, y_train, y_test = preprocess_data(df)

# Building Neural Network Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define a more complex architecture of your neural network with additional layers
def build_neural_network(input_dim):
    model = Sequential()

    # Add input layer
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.5))

    # Add hidden layers
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))

    # Add more hidden layers
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))

    # Add output layer with sigmoid activation for binary classification
    model.add(Dense(1, activation='sigmoid'))

    return model

# Get the number of features for input_dim
input_dim = X_train.shape[1]

# Build the even more complex neural network model
model = build_neural_network(input_dim)

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the model summary
print("Even More Complex Neural Network Model Summary:")
print(model.summary())


# Training the Model

Simple Training

In [ ]:
# Set the number of epochs and batch size
epochs = 10
batch_size = 64

# Train the model on the training data
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Display training history (loss and accuracy over epochs)
print("\nTraining History:")
print(history.history)

# Display test performance
print("\nTest Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Complex Training with
1.   EarlyStopping
2.   LearningRateScheduler



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# Define a learning rate schedule function (you can customize this based on your needs)
def lr_schedule(epoch):
    if epoch < 5:
        return 0.001
    elif epoch < 10:
        return 0.0005
    else:
        return 0.0001

# Set the number of epochs and batch size
epochs = 20
batch_size = 64

# Define early stopping to stop training if the validation loss doesn't improve
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Define learning rate scheduler
lr_scheduler = LearningRateScheduler(lr_schedule)

# Train the model on the training data with learning rate scheduling and early stopping
history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    callbacks=[early_stopping, lr_scheduler]
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Display training history (loss and accuracy over epochs)
print("\nTraining History:")
print(history.history)

# Display test performance
print("\nTest Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Evaluating the Model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predictions on the test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Convert y_test to binary format (0 or 1)
y_test_binary = y_test.astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary)
roc_auc = roc_auc_score(y_test_binary, y_pred)

# Display metrics
print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)

# Confusion Matrix
cm = confusion_matrix(y_test_binary, y_pred_binary)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, annot_kws={"size": 14})
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()


Fine Tunning the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Define a more advanced model
advanced_model = Sequential()
advanced_model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
advanced_model.add(BatchNormalization())
advanced_model.add(Dropout(0.5))
advanced_model.add(Dense(128, activation='relu'))
advanced_model.add(BatchNormalization())
advanced_model.add(Dropout(0.5))
advanced_model.add(Dense(64, activation='relu'))
advanced_model.add(BatchNormalization())
advanced_model.add(Dropout(0.5))
advanced_model.add(Dense(1, activation='sigmoid'))

# Compile the model with a lower learning rate and L2 regularization
advanced_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with early stopping
history_advanced = advanced_model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predictions on the test set
y_pred = advanced_model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Convert y_test to binary format (0 or 1)
y_test_binary = y_test.astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary)
roc_auc = roc_auc_score(y_test_binary, y_pred)

# Display metrics
print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)

# Confusion Matrix
cm = confusion_matrix(y_test_binary, y_pred_binary)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, annot_kws={"size": 14})
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()


L2 Regularization

In [ ]:
from tensorflow.keras.regularizers import l2

advanced_model = Sequential()
advanced_model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)))
advanced_model.add(BatchNormalization())
advanced_model.add(Dropout(0.5))
advanced_model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
advanced_model.add(BatchNormalization())
advanced_model.add(Dropout(0.5))
advanced_model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
advanced_model.add(BatchNormalization())
advanced_model.add(Dropout(0.5))
advanced_model.add(Dense(1, activation='sigmoid'))

# Compile the model
advanced_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predictions on the test set
y_pred = advanced_model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Convert y_test to binary format (0 or 1)
y_test_binary = y_test.astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary)
roc_auc = roc_auc_score(y_test_binary, y_pred)

# Display metrics
print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)


More tuning the improve the scores

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score

# Train the model with L2 regularization and early stopping
history_l2 = advanced_model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)

# Evaluate the model on the test set
y_pred_prob = advanced_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Adjust the threshold and calculate metrics
threshold = 0.3  # Experiment with different threshold values
y_pred_adjusted = (y_pred_prob > threshold).astype(int)

print("Classification Report:")
print(classification_report(y_test, y_pred_adjusted))

print("ROC AUC Score:", roc_auc_score(y_test, y_pred_prob))


Ensembling

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

# Assume you have already preprocessed X_train and y_train

# Define individual classifiers
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
lr_classifier = make_pipeline(StandardScaler(), LogisticRegression(random_state=42))
svm_classifier = make_pipeline(StandardScaler(), SVC(probability=True, random_state=42))

# Create a Voting Classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('random_forest', rf_classifier),
        ('logistic_regression', lr_classifier),
        ('svm', svm_classifier)
    ],
    voting='soft'  # Use soft voting for probability-based ensemble
)

# Evaluate the ensemble using cross-validation
y_pred = cross_val_predict(voting_classifier, X_train, y_train, cv=5, method='predict_proba')[:, 1]

# Threshold the probabilities to get binary predictions
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)

# Evaluate the performance
print("Ensemble Performance:")
print(classification_report(y_train, y_pred_binary))
print("Accuracy:", accuracy_score(y_train, y_pred_binary))


In [ ]:
pip install imbalanced-learn


Adding OverSampling

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

# Assume you have already preprocessed X_train and y_train

# Handle class imbalance with oversampling
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

# Define individual classifiers
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
lr_classifier = make_pipeline(StandardScaler(), LogisticRegression(random_state=42))
svm_classifier = make_pipeline(StandardScaler(), SVC(probability=True, random_state=42))

# Create a Voting Classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('random_forest', rf_classifier),
        ('logistic_regression', lr_classifier),
        ('svm', svm_classifier)
    ],
    voting='soft'  # Use soft voting for probability-based ensemble
)

# Evaluate the ensemble using cross-validation
y_pred = cross_val_predict(voting_classifier, X_resampled, y_resampled, cv=5, method='predict_proba')[:, 1]

# Threshold the probabilities to get binary predictions
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)

# Evaluate the performance
print("Ensemble Performance:")
print(classification_report(y_resampled, y_pred_binary))
print("Accuracy:", accuracy_score(y_resampled, y_pred_binary))


In [ ]:
saving the model

In [ ]:
from sklearn.ensemble import VotingClassifier
import joblib


# Save the ensemble model
joblib.dump(voting_classifier, 'ensemble_model.joblib')

# Save the preprocessor (ColumnTransformer)
joblib.dump(preprocessor, 'preprocessor.joblib')

# Save the numeric transformer (StandardScaler)
numeric_transformer = preprocessor.named_transformers_['num']['scaler']
joblib.dump(numeric_transformer, 'numeric_transformer.joblib')

# Save the categorical transformer (OneHotEncoder)
categorical_transformer = preprocessor.named_transformers_['cat']['onehot']
joblib.dump(categorical_transformer, 'categorical_transformer.joblib')

# Predicting New Data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import joblib

# Load the saved ensemble model
loaded_ensemble_model = joblib.load('ensemble_model.joblib')

# Load your new data for prediction (replace 'new_data.csv' with your file path)
new_data = pd.read_csv('trial.csv')

# Display the raw new data
print("Raw New Data:")
print(new_data.head())

# Assuming 'Default' column has missing values
# Add 'Default' column with NaN values
new_data['Default'] = np.nan

# Display the new data after adding 'Default' column
print("\nNew Data with 'Default' Column Added:")
print(new_data.head())


In [ ]:

# Drop 'LoanID' if present
if 'LoanID' in df.columns:
    df = df.drop('LoanID', axis=1)

import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assume 'new_data' is your DataFrame with the provided column names
# Replace 'NumericColumn' and 'CategoricalColumn' with actual column names

numeric_features = ['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio']
categorical_features = ['Education', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner']

# Create transformers for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first'))
])

# Create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply the preprocessing steps to new_data
X_new_data = preprocessor.fit_transform(new_data)


# Display the preprocessed new data
columns_after_onehot = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
all_columns = numeric_features + list(columns_after_onehot)
pd.DataFrame(X_new_data, columns=all_columns)


In [ ]:
# Check the column names in new_data
print(new_data.columns)

In [ ]:
# Make predictions using the loaded ensemble model
ensemble_model = VotingClassifier(
    estimators=[
        ('random_forest', rf_classifier),
        ('logistic_regression', lr_classifier),
        ('svm', svm_classifier)
    ],
    voting='soft'  # Use soft voting for probability-based ensemble
)

# Fit the ensemble model
ensemble_model.fit(X_train, y_train)

# Save the fitted ensemble model
joblib.dump(ensemble_model, 'ensemble_model.joblib')

# Now, you can load and use the fitted ensemble model for predictions
loaded_ensemble_model = joblib.load('ensemble_model.joblib')

# Make predictions
predictions = loaded_ensemble_model.predict(X_new_data)

# Display the predictions
print("\nPredictions:")
print(predictions)

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


# Manually input new data
new_data = {
    'Age': 29,
    'Income': 129670,
    'LoanAmount': 213947,
    'CreditScore': 593,
    'MonthsEmployed': 49,
    'NumCreditLines': 4,
    'InterestRate': 7.07,
    'LoanTerm': 60,
    'DTIRatio': 0.89,
    'Education': "Bachelor's",
    'MaritalStatus': "Single",
    'HasMortgage': "Yes",
    'HasDependents': "No",
    'LoanPurpose': "Business",
    'HasCoSigner': "No",
}

# Create a DataFrame with a single row using the new data
new_data_df = pd.DataFrame([new_data])

# Display the new data
print("New Data:")
print(new_data_df)

# Original preprocessing steps
numeric_features = ['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed',
                    'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio']

categorical_features = ['Education', 'MaritalStatus', 'HasMortgage', 'HasDependents',
                         'LoanPurpose', 'HasCoSigner']

# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', sparse=False))  # Set sparse to False
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Fit and transform the new data using the preprocessor
X_new_data = preprocessor.fit_transform(new_data_df)

# Get feature names after one-hot encoding
feature_names = numeric_features + list(preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features))

# Display the preprocessed new data
print("\nPreprocessed New Data:")
print(pd.DataFrame(X_new_data, columns=feature_names))

In [ ]:
import joblib

# Step 1: Load the saved model
loaded_ensemble_model = joblib.load("ensemble_model.joblib")

# Step 2: Preprocess the new data
X_new_data = preprocessor.transform(new_data_df)

# Step 3: Make predictions
predictions = loaded_ensemble_model.predict(X_new_data)

# Display the predictions
print("\nPredictions:")
print(predictions)
